### CNN with Image Augmentation (CIFAR10)
images of 10 categories: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. In RGB, 32x32 pixels. The state of the art accuracy is 96.53 percent.


In [1]:
# import tensorflow as tf
import numpy as np                                
# from tensorflow.examples.tutorials.mnist import input_data
import keras as keras 
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import (Dense, Dropout, Activation,
                                     Flatten)
from keras.applications.vgg16 import VGG16, preprocess_input

Using TensorFlow backend.


In [2]:
#load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_rows, img_cols , channels= 32,32,3

In [3]:
# set up image augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(x_train)

In [4]:
#reshape into images
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
input_shape = (img_rows, img_cols, 1)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
#convert integers to float; normalise and center the mean
x_train=x_train.astype("float32")  
x_test=x_test.astype("float32")

x_train = x_train/127.5 - 1.0
x_test = x_test/127.5 - 1.0

In [6]:
# labels
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
img_size = 32

## 如果include_top = True，shape不能自訂

In [8]:
vgg_model = VGG16(weights='imagenet', include_top=False,
                  input_shape=(img_size, img_size, 3))
# vgg_model.summary()

x = Flatten()(vgg_model.output)
x = Dense(128, activation='relu')(x)
# x = Dropout(0.25)(x)
outputs = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=vgg_model.inputs, outputs=outputs)
# model.summary()

In [9]:
vgg_model.trainable = True

trainable_layer = 3
for layer in vgg_model.layers[:-trainable_layer]:
    layer.trainable = False

for layer in model.layers:
    pass
#     print(layer, layer.trainable)


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(x_train, y_train, batch_size=640, epochs=8, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/8
50000/50000 [==============================] - 8s 165us/step - loss: 1.1553 - acc: 0.5939 - val_loss: 0.8904 - val_acc: 0.6870
Epoch 2/8
50000/50000 [==============================] - 5s 102us/step - loss: 0.7906 - acc: 0.7236 - val_loss: 0.8322 - val_acc: 0.7090
Epoch 3/8
50000/50000 [==============================] - 5s 102us/step - loss: 0.6837 - acc: 0.7600 - val_loss: 0.8026 - val_acc: 0.7223
Epoch 4/8
50000/50000 [==============================] - 5s 102us/step - loss: 0.6107 - acc: 0.7851 - val_loss: 0.7800 - val_acc: 0.7318
Epoch 5/8
50000/50000 [==============================] - 5s 103us/step - loss: 0.5393 - acc: 0.8087 - val_loss: 0.8341 - val_acc: 0.7208
Epoch 6/8
50000/50000 [==============================] - 5s 103us/step - loss: 0.4688 - acc: 0.8349 - val_loss: 0.8095 - val_acc: 0.7326
Epoch 7/8
50000/50000 [==============================] - 5s 104us/step - loss: 0.3989 - acc: 0.8586 - val_loss: 0.8613 - val_acc

In [10]:
model._name = 'vgg_cifar10'
print(model.name)
model.save('vgg_cifar10.h5') 

model_1


In [11]:
del model